# Introduction
The following code for NSE has been picked up from [Ewald's examples](https://github.com/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb) given below

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as st
from datetime import datetime
from itertools import product

from ib_insync import *
util.startLoop()

ib=IB()
ib.connect('127.0.0.1', 4004, clientId=12)

In [ ]:
# Standard Deviation calculation
call_probability = 0.97
put_probability = 0.92

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

# Get the list of Equity and Index Option scrips
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
paisa = pd.read_html(paisaurl, header=0)[1]          # It's the second table in the url

# Create iSymbol for the options
paisa["iSymbol"] = paisa.Symbol.str.slice(0,9)

# Replace & for M&M, L&T, etc. This is not needed for IBKR
mask = paisa.loc[paisa.Symbol.str.contains("&"), "iSymbol"]
paisa.loc[paisa.Symbol.isin(mask), "iSymbol"] = mask.str.replace("&", "")

# Add columns
paisa["Exchange"] = 'NSE'   # for the Exchange

# Exclusions and Inclusions
vix_nifty = ['VIX', 'NIFTY']     # mask for Index
symbol_rename = {'NIFTYCPSE': 'CPSE', 'NIFTY': 'NIFTY50', 'NIFTYMID5': 
                 'NIFTYMID50', 'NIFTYINFR': 'NIFTYINFRA'}   # renamed for IBKR
symbol_del = ['INDIAVIX']

# Clean up paisa
paisa_clean = paisa[['Symbol', 'Mlot', 'iSymbol', 'Exchange']]
paisa_clean = paisa_clean.loc[~paisa.Symbol.str.contains('|'.join(symbol_del)), :] # INDIAVIX not in IBKR
paisa_clean = paisa_clean.replace({"iSymbol": symbol_rename}) # Rename scrips to align with IBKR

# Stock dataframe
nse_stk = paisa_clean.loc[~index_mask, :].reset_index(drop=True) # Not the index mask!

# Qualify the Stock and add the contracts
qualStock = ib.qualifyContracts(*[Stock(x, y) for x, y in zip(nse_stk.iSymbol, nse_stk.Exchange)])
nse_stk["Contract"] = qualStock

########    Line below ignored because of inconsistency in index data #######
# # Index dataframe (Keep only VIX and NIFTY)
# index_mask = paisa_clean.Symbol.str.contains('|'.join(vix_nifty))
# nse_ind = paisa_clean.loc[index_mask, :].reset_index(drop=True)

# # Qualify the Index and add the contracts
# qualIndex = ib.qualifyContracts(*[Index(x, y) for x, y in zip(nse_ind.iSymbol, nse_ind.Exchange)])
# nse_ind["Contract"] = qualIndex #### Though successfully qualified tradingClass is different than iSymbol for index!!!

# # df_contracts = pd.concat([nse_stk, nse_ind]).reset_index(drop=True)
#############################################################################

df_contracts = nse_stk.reset_index(drop=True)

df_contracts = df_contracts[df_contracts.iSymbol.isin(["ADANIENT", "INFY"])] # !Data limiter

# Get the underlying's price
list_of_prices = [ib.reqTickers(i) for i in df_contracts.Contract]

# Make a dataframe of prices
u_price = pd.Series([i.close for elem in list_of_prices for i in elem])
df_contracts["U_Price"] = u_price.values

def get_chains(scrips):
    '''Gets the option chains for the scrips passed
       Args:
          (scrips): list: qualified list of equity or index options with conId
       Returns:
          list of option chain'''
    
    # Prepare a sequence for ib.reqSecDefOptParams
    seq = [(i.symbol, '', i.secType, i.conId) for i in scrips]

    # build the chains
    chains = [ib.reqSecDefOptParams(*p) for p in seq]
    # list(map(lambda p: ib.reqSecDefOptParams(*p), seq))  # Another way of doing!
    
    return chains

chains = get_chains(df_contracts.Contract)

# Build the dataframe for DTE and Std Deviation 
chains_for_sd = [(i.tradingClass, i.expirations) for elem in chains for i in elem]

# Cartesian for DTE (Ref: Stackoverflow: 51579138)
L = [[[x[0]], sorted(x[1])] for x in chains_for_sd]
df_dte = pd.DataFrame([j for i in L for j in product(*i)], columns=['iSymbol','Expiry'])

# function to get days to expiry
def get_dte(Expiry):
    '''Gives the expiry date
    Arg:
       (Expiry) = str of yyyymmdd format
    Returns:
       dte = int days to expiry'''
    exp_date = datetime.strptime(Expiry, '%Y%m%d')
    dte = (exp_date- datetime.now()).days
    return dte

# Get the DTE
df_dte["DTE"] = [get_dte(x) for x in df_dte.Expiry]

# Merge to prepare for standard deviation
df_sd = pd.merge(df_dte, df_contracts, on="iSymbol")

# function to get standard deviation
def get_stdev(contract, dte):
    '''Gets the Standard Deviation
    Args:
       (contract) = object: the qualified stock
       (dte)      = int: days to expiry
    Returns:
       standard deviation in days (int)
    '''
    sd_days = str(dte)+' D'
    bars = ib.reqHistoricalData(contract=contract, endDateTime='', durationStr=sd_days, 
                                barSizeSetting='1 day',  whatToShow='Trades', useRTH=True)
    sd = np.std([b.close for b in bars], ddof=1)
    return sd

df_sd['StDev'] = [get_stdev(x, y) for x, y in zip(df_sd.Contract, df_sd.DTE)] 
# list(map(lambda x, y: get_stdev(x, y), [i for i in df_sd.Contract], [j for j in df_sd.DTE])) # alternative

# Build dataframe for Strikes
chains_for_strikes = [(i.tradingClass, i.expirations, i.strikes) for elem in chains for i in elem]

# Cartesian for Strikes (Ref: Stackoverflow: 51579138)
M = [[[x[0]], sorted(x[1]), sorted(x[2])] for x in chains_for_strikes]
df_strikes = pd.DataFrame([j for i in M for j in product(*i)], columns=['iSymbol','Expiry', 'Strike'])

# Merge to get strikes and expiry
df_p = pd.merge(df_sd, df_strikes, on=["iSymbol", "Expiry"])

# Determine Put / Call rights
df_p["Right"] = ''
df_p.loc[df_p.Strike > (call_sd * df_p.StDev + df_p.U_Price), "Right"] = 'C'
df_p.loc[df_p.Strike <  df_p.U_Price - (put_sd * df_p.StDev), "Right"] = 'P'

# Get the focus list of Puts and Calls outside std deviation band
df_q = df_p.loc[df_p.Right != '', :].reset_index(drop=True)

In [ ]:
df = df_q.head()
df[["iSymbol", "Expiry", "Strike", "Right", "Mlot", "Exchange"]]

In [ ]:
Option()

In [ ]:
contract = Option([i for i in df.iSymbol], [i for i in df.Expiry], [i for i in df.Strike], [i for i in df.Right])

In [ ]:
ib.reqContractDetails(*contract)

In [ ]:
help(Option)

In [ ]:
c = Option(symbol='ADANIENT', exchange='NSE', 
                  lastTradeDateOrContractMonth='20181025', strike='155', right='P')

In [ ]:
order = Order(action='SELL', orderType='MKT', totalQuantity=4000)

In [ ]:
ib.whatIfOrder(c, order)

In [ ]:
x = [ib.reqTickers(c)]

In [ ]:
put_sd

In [ ]:
strikes = [strike for strike in df_q.Strike]
rights = [right for right in df_q.Right]
expirations = [expiration for expiration in df_q.Expiry]
symbols = [symbol for symbol in df_q.iSymbol]

contracts = [Option(symbol, expiration, strike, right, 'NSE')
            for symbol in symbols
            for expiration in expirations
            for strike in strikes
            for right in rights]


In [ ]:
contract = Option(symbol=scrip.symbol, exchange=scrip.exchange, 
                  lastTradeDateOrContractMonth='20180726', strike='640', right='P')
order = Order(action='SELL', orderType='MKT', totalQuantity=100.0)

In [ ]:
try:
    ib.whatIfOrder(contract, order)
except Exception as inst:
    print(inst.args)

In [ ]:
ib.whatIfOrder(contract, order)

# Original code of Ewald

In [ ]:
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)

In [ ]:
# Get the ticker
[ticker] = ib.reqTickers(spx)
ticker

In [ ]:
spxValue = ticker.marketPrice()

In [ ]:
spxValue

In [ ]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
util.df(chains)

In [ ]:
scrip = Stock('INTC', 'SMART', 'USD')
ib.qualifyContracts(scrip)

In [ ]:
scrip_chain = ib.reqSecDefOptParams(scrip.symbol, '', scrip.secType, scrip.conId)
util.df(scrip_chain)

In [ ]:
scrip_chain

In [ ]:
help(Contract.right)